In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2 as cv2
from PIL import Image
import os
import re
import json
from tqdm import tqdm

In [7]:
path = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/cropped/"
path_json = "/Users/satoshan/Library/CloudStorage/Box-Box/scan_org_charts/learning/output"
files = os.listdir(path)
files = [f for f in files if os.path.isfile(os.path.join(path, f))]

files_json = os.listdir(path_json)
files_json = [f for f in files_json if os.path.isfile(os.path.join(path_json, f))]

In [15]:
results = []

for i in tqdm(range(len(files)), desc="Processing files"):
    file = os.path.join(path, files[i])
    file_json = os.path.join(path_json, files_json[i])
    image = Image.open(file) 
    image_width, image_height = image.size 
    with open(file_json, "r") as f:
        data = json.load(f)
    # the coordation of the centers of department
    centers = []
    for i, box in enumerate(data["pred_boxes"]):
        x_center = int((box[0] + box[2]) / 2)
        y_center = int((box[1] + box[3]) / 2)

        # Normalize the image size
        x_normalized = (x_center / image_width) * 10
        y_normalized = (y_center / image_height) * 10
        centers.append({"id": i, "center": (x_normalized, y_normalized)})

    # make graph
    G = nx.Graph()
    # Add nodes to graph
    for center in centers:
        G.add_node(center["id"], pos=center["center"])
    # Add edges to graph 
    distance_threshold = 5
    for i in range(len(centers)):
        for j in range(i + 1, len(centers)):
            dist = np.linalg.norm(np.array(centers[i]["center"]) - np.array(centers[j]["center"]))
            if dist < distance_threshold:
                G.add_edge(centers[i]["id"], centers[j]["id"], weight=dist)

    pos = nx.get_node_attributes(G, "pos")
    shortest_length_path = nx.average_shortest_path_length(G)
    non_zero_boxes = [box for box in data["pred_boxes"] if any(coord != 0 for coord in box)]
    filtered_length = len(non_zero_boxes)
    match = re.search(r"cropped_(\d{4})_(\d{4})", file)
    if match:
        year = match.group(1)  # Extracts the year (e.g., "2002")
        code = match.group(2)  # Extracts the code (e.g., "7003")
    else:
        year = None
        code = None

    results.append({"code": code, "year": year , "shortest_path_length": shortest_length_path, "node_count": filtered_length})


Processing files: 100%|██████████| 101/101 [00:00<00:00, 104.90it/s]


In [27]:
i = 9
file_json = os.path.join(path_json, files_json[i])
with open(file_json, "r") as f:
    data = json.load(f)
non_zero_boxes = [box for box in data["pred_boxes"] if any(coord != 0 for coord in box)]
len(non_zero_boxes)

76

In [25]:
data["pred_boxes"]

[[561.1121826171875, 1989.7523193359375, 792.0862426757812, 2028.026611328125],
 [558.312255859375, 1846.2767333984375, 796.166748046875, 1880.703125],
 [560.9585571289062, 1134.32666015625, 795.1263427734375, 1171.1634521484375],
 [557.5484008789062, 1792.84228515625, 795.5090942382812, 1837.6064453125],
 [549.5059204101562, 2228.92822265625, 790.605712890625, 2273.4814453125],
 [558.7848510742188, 2137.010986328125, 786.4844360351562, 2173.295654296875],
 [559.4204711914062, 2038.322509765625, 784.32080078125, 2077.566162109375],
 [550.2691650390625, 2183.04638671875, 790.954345703125, 2224.97216796875],
 [99.8838119506836, 1144.4918212890625, 152.0628662109375, 1381.606201171875],
 [561.5822143554688, 1183.44384765625, 796.6010131835938, 1221.6541748046875],
 [847.3089599609375, 681.7322998046875, 895.261962890625, 957.7435913085938],
 [554.87109375, 1744.4766845703125, 799.1566772460938, 1788.7977294921875],
 [557.9364013671875, 1894.4466552734375, 794.490966796875, 1930.7172851562

In [16]:
df = pd.DataFrame(results)
df

,code,year,shortest_path_length,node_count
0,7273,2010,2.192982,76
1,7267,2010,1.295439,76
2,7298,2010,2.709825,76
3,7105,2010,1.521404,76
4,7313,2010,1.613333,76
...,...,...,...,...
96,7294,2010,1.889825,76
97,7280,2010,1.384211,76
98,7273,2002,1.307719,76
99,7242,2010,1.712982,76
